# Multimodal search using CLIP

![mmclip](https://www.researchgate.net/publication/363808556/figure/fig2/AS:11431281086053770@1664048343869/Architectures-of-the-designed-machine-learning-approaches-with-OpenAI-CLIP-model.jpg)

### Installing all dependencies

In [1]:
!pip install duckdb
!pip install gradio
!pip install transformers
!pip install pyarrow
!pip install argparse
!pip install lancedb
!pip install torch
!pip install torchvision
!pip install torchaudio

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


## First run setup: Download data and pre-process


In [8]:
import io
import PIL
import duckdb
import lancedb

## Create / Open LanceDB Table

In [3]:
!wget https://eto-public.s3.us-west-2.amazonaws.com/datasets/diffusiondb_lance.tar.gz
!tar -xvf diffusiondb_lance.tar.gz
!mv diffusiondb_test rawdata.lance

--2025-04-28 10:39:32--  https://eto-public.s3.us-west-2.amazonaws.com/datasets/diffusiondb_lance.tar.gz
Resolving eto-public.s3.us-west-2.amazonaws.com (eto-public.s3.us-west-2.amazonaws.com)... 3.5.80.150, 52.92.152.130, 52.92.248.234, ...
Connecting to eto-public.s3.us-west-2.amazonaws.com (eto-public.s3.us-west-2.amazonaws.com)|3.5.80.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6121107645 (5.7G) [application/x-gzip]
Saving to: ‘diffusiondb_lance.tar.gz’

diffusiondb_lance.t 100%[===================>]   5.70G  18.8MB/s    in 5m 43s  

2025-04-28 10:45:16 (17.0 MB/s) - ‘diffusiondb_lance.tar.gz’ saved [6121107645/6121107645]

diffusiondb_test/
diffusiondb_test/_versions/
diffusiondb_test/_latest.manifest
diffusiondb_test/data/
diffusiondb_test/data/138fc0d8-a806-4b10-84f8-00dc381afdad.lance
diffusiondb_test/_versions/1.manifest


In [12]:
!pip install --upgrade pyarrow
!pip install --upgrade pylance

In [13]:
import pyarrow.compute as pc
import lance

db = lancedb.connect("~/datasets/demo")
if "diffusiondb" in db.table_names():
    tbl = db.open_table("diffusiondb")
else:
    # First data processing and full-text-search index
    data = lance.dataset("rawdata.lance").to_table()
    # remove null prompts
    # tbl = db.create_table("diffusiondb", data.filter(~pc.field("prompt").is_null()), mode="overwrite") # OOM
    tbl = db.create_table("diffusiondb", data, mode="overwrite")
    tbl.create_fts_index(["prompt"])

## Create CLIP embedding function for the text

In [15]:
from transformers import CLIPModel, CLIPProcessor, CLIPTokenizerFast

MODEL_ID = "openai/clip-vit-base-patch32"

tokenizer = CLIPTokenizerFast.from_pretrained(MODEL_ID)
model = CLIPModel.from_pretrained(MODEL_ID)
processor = CLIPProcessor.from_pretrained(MODEL_ID)


def embed_func(query):
    inputs = tokenizer([query], padding=True, return_tensors="pt")
    text_features = model.get_text_features(**inputs)
    return text_features.detach().numpy()[0]

In [16]:
tbl.schema
# tbl.to_pandas().head() # OOM

prompt: string
seed: uint32
step: uint16
cfg: float
sampler: string
width: uint16
height: uint16
timestamp: timestamp[s]
image_nsfw: float
prompt_nsfw: float
vector: fixed_size_list<item: float>[512]
  child 0, item: float
image: binary


## Search functions for Gradio

In [17]:
def find_image_vectors(query):
    emb = embed_func(query)

    code = (
        "import lancedb\n"
        "db = lancedb.connect('~/datasets/demo')\n"
        "tbl = db.open_table('diffusiondb')\n\n"
        f"embedding = embed_func('{query}')\n"
        "tbl.search(embedding).limit(9).to_df()"
    )
    return (_extract(tbl.search(emb).limit(9).to_pandas()), code)


def find_image_keywords(query):
    code = (
        "import lancedb\n"
        "db = lancedb.connect('~/datasets/demo')\n"
        "tbl = db.open_table('diffusiondb')\n\n"
        f"tbl.search('{query}').limit(9).to_df()"
    )
    return (_extract(tbl.search(query).limit(9).to_pandas()), code)


def find_image_sql(query):
    code = (
        "import lancedb\n"
        "import duckdb\n"
        "db = lancedb.connect('~/datasets/demo')\n"
        "tbl = db.open_table('diffusiondb')\n\n"
        "diffusiondb = tbl.to_lance()\n"
        f"duckdb.sql('{query}').to_df()"
    )
    diffusiondb = tbl.to_lance()
    return (_extract(duckdb.sql(query).to_df()), code)


def _extract(df):
    image_col = "image"
    return [
        (PIL.Image.open(io.BytesIO(row[image_col])), row["prompt"])
        for _, row in df.iterrows()
    ]

## Setup Gradio interface

In [18]:
import gradio as gr


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Tab("Embeddings"):
            vector_query = gr.Textbox(value="portraits of a person", show_label=False)
            print('vector_query',vector_query)
            b1 = gr.Button("Submit")
        with gr.Tab("Keywords"):
            keyword_query = gr.Textbox(value="ninja turtle", show_label=False)
            b2 = gr.Button("Submit")
        with gr.Tab("SQL"):
            sql_query = gr.Textbox(
                value="SELECT * from diffusiondb WHERE image_nsfw >= 2 LIMIT 9",
                show_label=False,
            )
            b3 = gr.Button("Submit")
    with gr.Row():
        code = gr.Code(label="Code", language="python")
    with gr.Row():
        gallery = gr.Gallery(
            label="Found images", show_label=False, elem_id="gallery"
        ).style(columns=[3], rows=[3], object_fit="contain", height="auto")

    b1.click(find_image_vectors, inputs=vector_query, outputs=[gallery, code])
    b2.click(find_image_keywords, inputs=keyword_query, outputs=[gallery, code])
    b3.click(find_image_sql, inputs=sql_query, outputs=[gallery, code])


demo.launch(share=True)

<ipython-input-18-b0f437326cca>:23: GradioDeprecationWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  ).style(columns=[3], rows=[3], object_fit="contain", height="auto")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.41.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://0aee4ad1e9728a45ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
